In [0]:
from pyspark.sql.functions import lit

CATALOG_NAME = "saude_global"
TABLE_SILVER = f"{CATALOG_NAME}.silver.daily_health_metrics"
TABLE_GOLD_DIM_COUNTRY = f"{CATALOG_NAME}.gold.dim_country"
TABLE_GOLD_DIM_DATE = f"{CATALOG_NAME}.gold.dim_date"
TABLE_GOLD_FACT = f"{CATALOG_NAME}.gold.fact_daily_metrics"

spark.sql(f"""
    CREATE OR REPLACE TABLE {TABLE_GOLD_DIM_COUNTRY}
    USING DELTA
    AS
    SELECT 
        DISTINCT 
        iso_code,
        country_name,
        continent_clean AS continent
    FROM {TABLE_SILVER}
    WHERE iso_code IS NOT NULL
    ORDER BY country_name;
""")

spark.sql(f"""
    CREATE OR REPLACE TABLE {TABLE_GOLD_DIM_DATE}
    USING DELTA
    AS
    SELECT
        DISTINCT
        T1.report_date AS date_key,
        YEAR(T1.report_date) AS year,
        MONTH(T1.report_date) AS month,
        DAYOFMONTH(T1.report_date) AS day_of_month,
        DATE_FORMAT(T1.report_date, 'MMMM') AS month_name,
        WEEKDAY(T1.report_date) AS day_of_week_num,
        CASE WHEN WEEKDAY(T1.report_date) IN (5, 6) THEN 'Weekend' ELSE 'Weekday' END AS is_weekend
    FROM {TABLE_SILVER} AS T1
    WHERE T1.report_date IS NOT NULL
    ORDER BY date_key;
""")

spark.sql(f"""
    CREATE OR REPLACE TABLE {TABLE_GOLD_FACT}
    USING DELTA
    AS
    SELECT
        iso_code,                   
        report_date AS date_key,    
        daily_new_cases,
        daily_new_deaths,
        daily_vaccinations,
        case_incidence_per_100k
    FROM {TABLE_SILVER}
    WHERE report_date IS NOT NULL;
""")

spark.sql(f"""
    OPTIMIZE {TABLE_GOLD_FACT}
    ZORDER BY (iso_code, date_key);
""")
spark.sql(f"OPTIMIZE {TABLE_GOLD_DIM_COUNTRY};")
spark.sql(f"OPTIMIZE {TABLE_GOLD_DIM_DATE};")
